In [1]:
import nltk
from nltk.stem.porter import PorterStemmer
import numpy as np
import json
import string
stemmer = PorterStemmer()

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [3]:
import contractions

In [4]:
def tokenize(sentence):
    expanded_text = []
    for word in sentence.split():
        expanded_text.append(contractions.fix(word))
    return nltk.word_tokenize("".join(expanded_text))
def stem(word):
    return stemmer.stem(word.lower())
def bag_of_words(tokenized_sentence, all_words):
    tokenized_sentence = [stem(w) for w in tokenized_sentence]
    bag = np.zeros(len(all_words), dtype=np.float32)
    for idx, w in enumerate(all_words):
        if w in tokenized_sentence:
            bag[idx] = 1.0
    return bag

In [5]:
with open('intent.json') as f:
    intents = json.load(f)

In [6]:
all_words = []
tags = []
xy = []

for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w,tag))

ignore_words = list(string.punctuation)

In [7]:
all_words = [stem(w) for w in all_words if w not in ignore_words]

In [8]:
all_words = sorted(set(all_words))
tags = sorted(set(tags))

In [9]:
x_train = []
y_train = []

for (pattern_sent,tag) in xy:
    bag = bag_of_words(pattern_sent, all_words)
    x_train.append(bag)
    label = tags.index(tag)
    y_train.append(label)

In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)

In [ ]:
y_train

In [10]:
class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(x_train)
        self.x_data = x_train
        self.y_data = y_train

    #dataset [idx]
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.n_samples

batch_size = 8
hidden_size = 8
input_size =  len(x_train[0])
output_size = len(tags) 
learning_rate = 0.001
num_epochs = 1000

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=0)



class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet,self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):

        out = self.l1(x) 
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # по activation and no softmax
        return out
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:

model = NeuralNet(input_size, hidden_size, output_size)

# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.long().to(device)
        # labels = labels.to(device)
    # forward
        outputs = model(words)
        loss = criterion(outputs, labels)
    
    #backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch +1 ) % 100 == 0:
        print(f'epoch {epoch+1}/{num_epochs}, loss = {loss.item():.4f}')
print(f'final loss = {loss.item():.4f}')

In [ ]:
data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "output_size": output_size,
    "hiddent_size": hidden_size,
    "all_words": all_words,
    "tags": tags
 }

FILE = "data.pth"
torch.save(data,FILE)

In [11]:
import random
data = torch.load("data.pth")

input_size = data["input_size"]
hidden_size = data["hiddent_size"]
output_size = data["output_size"]
all_words = data["all_words"]
tags = data["tags"]
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model. load_state_dict(model_state)
model.eval()


NeuralNet(
  (l1): Linear(in_features=72, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=9, bias=True)
  (relu): ReLU()
)

In [12]:
import time

In [13]:
bot_name = "Bot"
print("Let's Chat with a bot and type 'quit' to exit")
while True:
    sentence = input("User: ")
    print("User :- ", sentence)
    time.sleep(1)
    if sentence == "quit":
        break
    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X)

    output = model(X)
    _, predicted = torch.max(output, dim=1)
    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]

    if prob.item() > 0.75:
        for intent in intents["intents"]:
            if tag == intent["tag"]:
                print(f"{bot_name} : {random.sample(intent['responses'],3)}")
                time.sleep(1)
    else:
        print(f"{bot_name} : I do not understand...")
        time.sleep(1)

Let's Chat with a bot and type 'quit' to exit
User :-  Hello
Bot : ["I'm doing well, thanks for asking. How can I assist you?", 'Hello, thanks for asking', 'Good to see you again']
User :-  Stop
Bot : ["I understand that you're frustrated. Is there anything I can do to help address your concerns?", "I'm sorry if I said anything that upset you. Is there anything specific that I can help you with?", 'I apologize if my messages are bothering you. If you would like me to stop messaging you, please just let me know.']
User :-  stop now
Bot : I do not understand...
User :-  quit
